# TFM.

## Regresión Logística con SKLearn

In [1]:
import pyodbc

server = 'BIOE-MAJC' 
database = 'bioe' 
username = 'bioe' 
password = 'oe@LoCaL' 
trusted = 'yes;' 

cnxn = pyodbc.connect('Driver={SQL Server};SERVER='+server+';DATABASE='+database+';ENCRYPT=no;'+username+';PWD='+ password)
cursor = cnxn.cursor()

import pandas as pd


#### Algoritmo Regresión Logística

In [2]:
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

query = "SELECT dG_S,benchmark, CP_BFKV,CP_BL,CP_BT,CP_GKS,CP_HLPL,CP_MJPL,CP_MJ3h,CP_MJ2h,CP_MJ1,CP_MJ2,CP_MSBM,CP_MS,CP_Qa,CP_Qm,CP_Qp" + \
  ",CP_RO,CP_SKOb,CP_SKOa,CP_SJKG,CP_TD,CP_TEl,CP_TEs,CP_TS,CP_VD,CP_SKOIP,AP_DCOMPLEX,AP_dDFIRE,AP_DFIRE2,CP_RMFCEN1" + \
  ",CP_RMFCEN2,CP_RMFCA,CP_TB,CP_TSC,AP_T1,AP_T2,AP_DOPE,AP_DOPE_HR,AP_ACE,INSIDE,HBOND,PI_PI,CAT_PI,ALIPH,ZRANK,ZRANK2" + \
  ",ROT_S,TRANS_S,NIPacking,NSC,FA_ATR,FA_REP,LK_SOLV,FA_PP,CG_VDW,CG_PP,CG_ENV,CG_BETA,HBOND2,AA_PROP,ROSETTADOCK,NHB" + \
  ",ELE,DESOLV,VDW,PYDOCK_TOT,ODA,PROPNSTS,SIPPER,AP_OPUS_PSP,AP_GEOMETRIC,AP_DARS,AP_URS,AP_MPS,AP_W1,CP_D1,AP_calRW" + \
  ",AP_calRWp,AP_PISA,FIREDOCK,FIREDOCK_AB,FIREDOCK_EI,CP_PIE,CP_DDG_U,CP_DDG_W,AP_DDG_U,AP_DDG_W,DDG_V FROM [bioe].[TFM].[tfm_NNaN0_S] ();"

df = pd.read_sql_query(query, cnxn)
print(df.head(5))
#print(df.loc[0,:])
#print(df[["CP_BFKV","CP_BL"]])


      Complex  benchmark  dG_S  CP_BFKV  CP_BL  CP_BT  CP_GKS  CP_HLPL  \
0   1A2K_C:AB          1     1  -0.8654 -6.506  -5.48    -8.0    -28.6   
1    1ACB_E:I          1     1  -2.3756 -6.130  -7.74    -2.4    -34.0   
2   1AHW_AB:C          1     1  -1.0114 -8.650  -4.96   -14.0    -34.2   
3    1AK4_A:D          1     0   1.1344 -2.076  -3.98    -4.0    -12.0   
4  1AKJ_AB:DE          1     0   1.5450 -4.558   2.74   -10.0    -26.8   

   CP_MJPL  CP_MJ3h  ...  AP_PISA  FIREDOCK  FIREDOCK_AB  FIREDOCK_EI  \
0   -140.8    -2.96  ...  -0.2445    -58.02       -71.00       -25.76   
1   -178.0   -11.50  ...  -0.4878    -88.88       -96.94       -62.16   
2   -196.2     2.36  ...  -0.0288    -36.54       -67.22        -3.33   
3   -122.4    -2.90  ...  -0.2250    -42.73       -47.67       -29.09   
4   -191.2     4.64  ...  -0.2936    -45.19       -67.13        -4.81   

     CP_PIE   CP_DDG_U   CP_DDG_W   AP_DDG_U   AP_DDG_W      DDG_V  
0  1.138510  -4.560318  -3.938181  -2.889758 -1

C:\Users\coept\AppData\Local\Temp\ipykernel_20388\2632812162.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, cnxn)


In [3]:
X_train, y_train = df.loc[df["benchmark"] == 1][["CP_BFKV","CP_BL","CP_BT","CP_GKS","CP_HLPL","CP_MJPL","CP_MJ3h","CP_MJ2h","CP_MJ1","CP_MJ2","CP_MSBM","CP_MS","CP_Qa","CP_Qm","CP_Qp" \
  ,"CP_RO","CP_SKOb","CP_SKOa","CP_SJKG","CP_TD","CP_TEl","CP_TEs","CP_TS","CP_VD","CP_SKOIP","AP_DCOMPLEX","AP_dDFIRE","AP_DFIRE2","CP_RMFCEN1" \
  ,"CP_RMFCEN2","CP_RMFCA","CP_TB","CP_TSC","AP_T1","AP_T2","AP_DOPE","AP_DOPE_HR","AP_ACE","INSIDE","HBOND","PI_PI","CAT_PI","ALIPH","ZRANK","ZRANK2" \
  ,"ROT_S","TRANS_S","NIPacking","NSC","FA_ATR","FA_REP","LK_SOLV","FA_PP","CG_VDW","CG_PP","CG_ENV","CG_BETA","HBOND2","AA_PROP","ROSETTADOCK","NHB" \
  ,"ELE","DESOLV","VDW","PYDOCK_TOT","ODA","PROPNSTS","SIPPER","AP_OPUS_PSP","AP_GEOMETRIC","AP_DARS","AP_URS","AP_MPS","AP_W1","CP_D1","AP_calRW" \
  ,"AP_calRWp","AP_PISA","FIREDOCK","FIREDOCK_AB","FIREDOCK_EI","CP_PIE","CP_DDG_U","CP_DDG_W","AP_DDG_U","AP_DDG_W","DDG_V"]], df.loc[df["benchmark"] == 1]["dG_S"]
X_test, y_test = df.loc[df["benchmark"] == 2][["CP_BFKV","CP_BL","CP_BT","CP_GKS","CP_HLPL","CP_MJPL","CP_MJ3h","CP_MJ2h","CP_MJ1","CP_MJ2","CP_MSBM","CP_MS","CP_Qa","CP_Qm","CP_Qp" \
  ,"CP_RO","CP_SKOb","CP_SKOa","CP_SJKG","CP_TD","CP_TEl","CP_TEs","CP_TS","CP_VD","CP_SKOIP","AP_DCOMPLEX","AP_dDFIRE","AP_DFIRE2","CP_RMFCEN1" \
  ,"CP_RMFCEN2","CP_RMFCA","CP_TB","CP_TSC","AP_T1","AP_T2","AP_DOPE","AP_DOPE_HR","AP_ACE","INSIDE","HBOND","PI_PI","CAT_PI","ALIPH","ZRANK","ZRANK2" \
  ,"ROT_S","TRANS_S","NIPacking","NSC","FA_ATR","FA_REP","LK_SOLV","FA_PP","CG_VDW","CG_PP","CG_ENV","CG_BETA","HBOND2","AA_PROP","ROSETTADOCK","NHB" \
  ,"ELE","DESOLV","VDW","PYDOCK_TOT","ODA","PROPNSTS","SIPPER","AP_OPUS_PSP","AP_GEOMETRIC","AP_DARS","AP_URS","AP_MPS","AP_W1","CP_D1","AP_calRW" \
  ,"AP_calRWp","AP_PISA","FIREDOCK","FIREDOCK_AB","FIREDOCK_EI","CP_PIE","CP_DDG_U","CP_DDG_W","AP_DDG_U","AP_DDG_W","DDG_V"]], df.loc[df["benchmark"] == 2]["dG_S"]


X = X_train
y = y_train
X.shape


(143, 87)

In [7]:
model = linear_model.LogisticRegression(C=0.1,max_iter=10000)
model.fit(X,y)


LogisticRegression(C=0.1, max_iter=10000)

In [9]:
predictions = model.predict(X)
print(predictions)


[1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 0 1 1 1 0
 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 0 1 1
 1 0 0 1 0 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 1 1 1 0 1 1 1
 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0 1]


In [10]:
model.score(X,y)


0.958041958041958

In [11]:
predictions = model.predict(X_test)
print(predictions)
model.score(X_test,y_test)


[1 0 0 1 1 1 0 0 0 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0]


0.5714285714285714

In [16]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10)
cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(msg)


Logistic Regression: 0.636190 (0.137836)


In [17]:
predictions = model.predict(X_test)
print(accuracy_score(y_test, predictions))


0.5714285714285714


In [19]:
print(confusion_matrix(y_test, predictions))


[[ 5  7]
 [ 8 15]]


In [20]:
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.38      0.42      0.40        12
           1       0.68      0.65      0.67        23

    accuracy                           0.57        35
   macro avg       0.53      0.53      0.53        35
weighted avg       0.58      0.57      0.58        35

